# Install and load package

In [3]:
% pip install transformers flask beautifulsoup4 requests nltk

     ---------------------------------------- 0.0/143.0 kB ? eta -:--:--
     ------- ----------------------------- 30.7/143.0 kB 660.6 kB/s eta 0:00:01
     ---------- -------------------------- 41.0/143.0 kB 393.8 kB/s eta 0:00:01
     --------------- --------------------- 61.4/143.0 kB 656.4 kB/s eta 0:00:01
     ------------------------------ ----- 122.9/143.0 kB 722.1 kB/s eta 0:00:01
     --------------------------------- -- 133.1/143.0 kB 714.4 kB/s eta 0:00:01
     ------------------------------------ 143.0/143.0 kB 606.1 kB/s eta 0:00:00
   ---------------------------------------- 0.0/99.7 kB ? eta -:--:--
   -------------------------------- ------- 81.9/99.7 kB 2.3 MB/s eta 0:00:01
   ---------------------------------------- 99.7/99.7 kB 1.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
! pip install Sastrawi

  Using cached Sastrawi-1.0.1-py2.py3-none-any.whl (209 kB)


In [2]:
! pip install accelerate -U

  Using cached accelerate-0.25.0-py3-none-any.whl.metadata (18 kB)
  Using cached huggingface_hub-0.19.4-py3-none-any.whl.metadata (14 kB)
Using cached accelerate-0.25.0-py3-none-any.whl (265 kB)
   ---------------------------------------- 0.0/277.5 kB ? eta -:--:--
   ---- ----------------------------------- 30.7/277.5 kB 1.3 MB/s eta 0:00:01
   ----- --------------------------------- 41.0/277.5 kB 487.6 kB/s eta 0:00:01
   -------------------- ------------------- 143.4/277.5 kB 1.1 MB/s eta 0:00:01
   ----------------------------------- ---- 245.8/277.5 kB 1.4 MB/s eta 0:00:01
   ---------------------------------------- 277.5/277.5 kB 1.6 MB/s eta 0:00:00
Using cached huggingface_hub-0.19.4-py3-none-any.whl (311 kB)


In [4]:
import numpy as np
import pandas as pd
import nltk
import random
import json
import re
import string
import itertools

from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, StopWordRemover, ArrayDictionary

nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\wilda\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Load data

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
with open('datasets/intent_datasets.json') as file:
    intents = json.load(file)

# extract data to dataframe

In [6]:
df = pd.DataFrame(intents['intents'])
df

,tag,patterns,responses
0,salam,"[Hai, Halo, Hola, Helo, hallo, hello, Ola]",[Halo. Ceritakan bagaimana perasaanmu hari ini...
1,tanya keberadaan,"[Apakah ada orang di sana?, ada orang di sana?...","[Halo. saya disini, ada yang bisa aku bantu ?,..."
2,pagi,"[pagi, Selamat pagi, halo Selamat pagi, hai Se...",[Selamat pagi. Saya harap tidur malam Anda nye...
3,siang,"[siang, Selamat siang, halo Selamat siang, hai...","[Selamat siang. Bagaimana harimu?, Selamat sia..."
4,malam,"[malam, Selamat malam, halo Selamat malam, hai...","[Selamat malam. Tidur yang nyenyak, Selamat ma..."
...,...,...,...
572,mental health tanya 11,[bagaimana cara menjaga kesehatan mental saat ...,[Menjaga kesehatan di tengah cobaan hidup adal...
573,mental health tanya 12,[bagaimana cara mengatur pikiran agar mental t...,[Relaksasi sangat penting dalam mengelola kese...
574,mental health tanya 13,[Gimana orang awam yang bukan psikolog atau ex...,"[Ketika seseorang merasa insecure, ada potensi..."
575,mental health tanya 14,[bagaimana cara agar mental kita kuat ?],[Kekuatan mental yang kuat merupakan kunci dal...


In [7]:
dic = {"tag":[], "patterns":[], "responses":[]}
for i in range(len(df)):
    ptrns = df[df.index == i]['patterns'].values[0]
    rspns = df[df.index == i]['responses'].values[0]
    tag = df[df.index == i]['tag'].values[0]
    for j in range(len(ptrns)):
        dic['tag'].append(tag)
        dic['patterns'].append(ptrns[j])
        dic['responses'].append(rspns)

df = pd.DataFrame.from_dict(dic)
df

,tag,patterns,responses
0,salam,Hai,[Halo. Ceritakan bagaimana perasaanmu hari ini...
1,salam,Halo,[Halo. Ceritakan bagaimana perasaanmu hari ini...
2,salam,Hola,[Halo. Ceritakan bagaimana perasaanmu hari ini...
3,salam,Helo,[Halo. Ceritakan bagaimana perasaanmu hari ini...
4,salam,hallo,[Halo. Ceritakan bagaimana perasaanmu hari ini...
...,...,...,...
1911,mental health tanya 11,bagaimana cara menjaga kesehatan mental saat i...,[Menjaga kesehatan di tengah cobaan hidup adal...
1912,mental health tanya 12,bagaimana cara mengatur pikiran agar mental te...,[Relaksasi sangat penting dalam mengelola kese...
1913,mental health tanya 13,Gimana orang awam yang bukan psikolog atau exp...,"[Ketika seseorang merasa insecure, ada potensi..."
1914,mental health tanya 14,bagaimana cara agar mental kita kuat ?,[Kekuatan mental yang kuat merupakan kunci dal...


In [8]:
# show dataset information
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1916 entries, 0 to 1915
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   tag        1916 non-null   object
 1   patterns   1916 non-null   object
 2   responses  1916 non-null   object
dtypes: object(3)
memory usage: 45.0+ KB


In [9]:
# display descriptive statistics of the dataset
df.describe()

,tag,patterns,responses
count,1916,1916,1916
unique,577,1898,548
top,kematian,Apakah masih banyak orang yang tidak bisa kelu...,"[Saya dapat memahami perasaan Anda. Pertama, p..."
freq,10,2,14


In [10]:
# display the number of null data in the dataset
df.isnull().sum()

tag          0
patterns     0
responses    0
dtype: int64

# Data Visualization

In [11]:
import plotly.graph_objects as go

intent_counts = df['tag'].value_counts()
fig = go.Figure(data=[go.Bar(x=intent_counts.index, y=intent_counts.values)])
fig.update_layout(title='Distribution of Intents', xaxis_title='Intents', yaxis_title='Count')
fig.show()

In [12]:
df['pattern_count'] = df['patterns'].apply(lambda x: len(x))
df['response_count'] = df['responses'].apply(lambda x: len(x))
avg_pattern_count = df.groupby('tag')['pattern_count'].mean()
avg_response_count = df.groupby('tag')['response_count'].mean()

fig = go.Figure()
fig.add_trace(go.Bar(x=avg_pattern_count.index, y=avg_pattern_count.values, name='Average Pattern Count'))
fig.add_trace(go.Bar(x=avg_response_count.index, y=avg_response_count.values, name='Average Response Count'))
fig.update_layout(title='Pattern and Response Analysis', xaxis_title='Intents', yaxis_title='Average Count')
fig.show()

# Data Preprocessing

## Make preprocessing function

In [13]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()
stopWordFactory = StopWordRemoverFactory()

In [14]:
# membuat text menjadi huruf kecil semua
def lower(text):
    return text.lower()

# replace text yg bukan termasuk ascii
def non_ascii(text):
    return text.encode('ascii', 'replace').decode('ascii')

# hapus tanda baca
def remove_punctuation(text):
    remove = string.punctuation
    remove = remove.replace("_", "")
    pattern = r"[{}]".format(remove)
    return re.sub(pattern, "", text)

# hapus spasi diawal dan akhir
def remove_whitespace_LT(text):
    return text.strip()

# hapus spasi double
def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

# hapus tab
def remove_tab(text):
    return text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")

# hapus multiple tab
def remove_tab2(text):
    return re.sub('\s+',' ',text)

# hapus angka
def remove_angka(text):
    return re.sub(r"\d+", "", text)

# hapus emoji
def remove_emoji(text):
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

# hapus titik banyak (...)
def remove_excessive_dot(text):
    return text.replace('..'," ")

# stopwords
def stopwords(text):
    more_stopword = ['sih', 'nya','rt','loh','lah', 'dd', 'mah', 'nye', 'eh', 'ehh', 'ah', 'yang','yg']
    data = stopWordFactory.get_stop_words()
    data.remove('tidak')
    stopwords_sastrawi = stopWordFactory.create_stop_word_remover()

    dictionary = ArrayDictionary(data+more_stopword)
    str_stopwords = StopWordRemover(dictionary)
    return str_stopwords.remove(text)

# stemming
def stemming(text):
    return stemmer.stem(text)

In [15]:
# slice dictionary
def slice_dict(data,total):
    return dict(itertools.islice(data.items(), total))

## Text Normalization

In [16]:
indo_slang_word = pd.read_csv("https://raw.githubusercontent.com/nasalsabila/kamus-alay/master/colloquial-indonesian-lexicon.csv")
indo_slang_word.head()

,slang,formal,In-dictionary,context,category1,category2,category3
0,woww,wow,1,wow,elongasi,0,0
1,aminn,amin,1,Selamat ulang tahun kakak tulus semoga panjang...,elongasi,0,0
2,met,selamat,1,Met hari netaas kak!? Wish you all the best @t...,abreviasi,0,0
3,netaas,menetas,1,Met hari netaas kak!? Wish you all the best @t...,afiksasi,elongasi,0
4,keberpa,keberapa,0,Birthday yg keberpa kak?,abreviasi,0,0


In [17]:
indo_slang_word.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15006 entries, 0 to 15005
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   slang          15006 non-null  object
 1   formal         15006 non-null  object
 2   In-dictionary  15006 non-null  int64 
 3   context        15006 non-null  object
 4   category1      15006 non-null  object
 5   category2      15006 non-null  object
 6   category3      15006 non-null  object
dtypes: int64(1), object(6)
memory usage: 820.8+ KB


In [18]:
# convert df slang to dict
df_slang = indo_slang_word[['slang','formal']]
slang_dict = dict(df_slang.values)
# slang_dict[:5]

In [19]:
# function to replace slang word
def replace_slang_word(string):
    new_string = " ".join(slang_dict.get(word, word) for word in string.split(' '))
    return new_string

In [20]:
# example
kata = "km sdh makan met bobo ya jgn nyebelin"
hasil = replace_slang_word(kata)
hasil

'kamu sudah makan selamat bobo ya jangan menyebalkan'

## Replace text

In [21]:
for text in df['patterns'].values:
    proc = lower(text)
    proc = non_ascii(proc)
    proc = remove_emoji(proc)
    proc = remove_tab(proc)
    proc = remove_tab2(proc)
    proc = remove_punctuation(proc)
    proc = remove_excessive_dot(proc)
    proc = remove_angka(proc)
    proc = remove_whitespace_LT(proc)
    proc = remove_whitespace_multiple(proc)
    proc = stopwords(proc)
    proc = stemming(proc)
    df['patterns'] = df['patterns'].replace(text, proc)

In [22]:
df.tail(10)

,tag,patterns,responses,pattern_count,response_count
1906,mental health tanya 6,masih banyak orang tidak keluar lingkung tidak...,"[Banyak, Banyak sekali orang yang merasakan ha...",95,1
1907,mental health tanya 7,giat apa bisa kurang stres mudah laku kapan saja,[Anda dapat mencoba teknik pernapasan yang efe...,84,1
1908,mental health tanya 8,bagaimana orang awam bukan psikolog tau kalau ...,"[Ketika seseorang merasa insecure, ada potensi...",94,1
1909,mental health tanya 9,cara kuat mental,[Kekuatan mental yang kuat merupakan kunci dal...,21,1
1910,mental health tanya 10,apa buat isu mental health makin parah,[Faktor-faktor yang dapat memperburuk kesehata...,55,1
1911,mental health tanya 11,bagaimana cara jaga sehat mental ini,[Menjaga kesehatan di tengah cobaan hidup adal...,50,1
1912,mental health tanya 12,bagaimana cara atur pikir mental tetap jaga,[Relaksasi sangat penting dalam mengelola kese...,59,1
1913,mental health tanya 13,gimana orang awam bukan psikolog expert tau ka...,"[Ketika seseorang merasa insecure, ada potensi...",125,1
1914,mental health tanya 14,bagaimana cara mental kuat,[Kekuatan mental yang kuat merupakan kunci dal...,38,1
1915,mental health tanya 15,bagaimana cara keluar lingkung tidak sehat mental,"[Banyak, Banyak sekali orang yang merasakan ha...",74,1


## Split data patterns and responses

In [23]:
df_patterns = df[['tag','patterns']]
df_patterns.head()

,tag,patterns
0,salam,hai
1,salam,halo
2,salam,hola
3,salam,helo
4,salam,hallo


In [24]:
df_responses = df[['tag','responses']]
df_responses.head()

,tag,responses
0,salam,[Halo. Ceritakan bagaimana perasaanmu hari ini...
1,salam,[Halo. Ceritakan bagaimana perasaanmu hari ini...
2,salam,[Halo. Ceritakan bagaimana perasaanmu hari ini...
3,salam,[Halo. Ceritakan bagaimana perasaanmu hari ini...
4,salam,[Halo. Ceritakan bagaimana perasaanmu hari ini...


In [25]:
df_patterns.shape

(1916, 2)

In [26]:
df_patterns.describe()

,tag,patterns
count,1916,1916
unique,577,1811
top,kematian,aku takut dunia
freq,10,5


## Data Balancing with Oversampling

In [27]:
df_patterns['tag'].value_counts()

tag
kematian                    10
stres                       10
kerja terhambat keluarga    10
semangat belajar            10
baik                         9
                            ..
membagi waktu mental         1
fakta-27                     1
lingkungan pintar            1
hidup tempat baru            1
mental health tanya 15       1
Name: count, Length: 577, dtype: int64

In [28]:
# menyeimbangkan kelas-kelas yang tidak seimbang dengan menduplikasi baris-baris dari kelas-kelas tersebut secara acak
def balance_data(df_patterns):
    df_tag = df_patterns['tag']
    max_counts = df_tag.value_counts().max()

    new_df = df_patterns.copy()
    for i in df_tag.unique():
        i_count = int(df_tag[df_tag == i].value_counts())
        if i_count < max_counts:
            i_samples = df_patterns[df_tag == i].sample(max_counts - i_count, replace = True, ignore_index = True)
            new_df = pd.concat([new_df, i_samples])
    return new_df

In [29]:
df_patterns = balance_data(df_patterns)

C:\Users\wilda\AppData\Local\Temp\ipykernel_13812\4142338773.py:8: FutureWarning:

Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead

C:\Users\wilda\AppData\Local\Temp\ipykernel_13812\4142338773.py:8: FutureWarning:

Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead

C:\Users\wilda\AppData\Local\Temp\ipykernel_13812\4142338773.py:8: FutureWarning:

Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead

C:\Users\wilda\AppData\Local\Temp\ipykernel_13812\4142338773.py:8: FutureWarning:

Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead

C:\Users\wilda\AppData\Local\Temp\ipykernel_13812\4142338773.py:8: FutureWarning:

Calling int on a single element Series is deprecated and will raise a TypeError in th

In [30]:
df_patterns['tag'].value_counts()

tag
salam                     10
pintar mereka             10
bingung topik skripsi     10
takut dunia               10
takut akhirat             10
                          ..
minat hilang              10
minat dan mood            10
beban orang tua           10
cara hidup mandiri        10
mental health tanya 15    10
Name: count, Length: 577, dtype: int64

In [31]:
df_patterns.count()

tag         5770
patterns    5770
dtype: int64

## Replace labels string by label number

In [32]:
labels = df_patterns['tag'].unique().tolist()
labels = [s.strip() for s in labels]
labels[:5]

['salam', 'tanya keberadaan', 'pagi', 'siang', 'malam']

In [33]:
num_labels = len(labels)
id2label = {id:label for id, label in enumerate(labels)}
label2id = {label:id for id, label in enumerate(labels)}

In [34]:
slice_dict(label2id,5)

{'salam': 0, 'tanya keberadaan': 1, 'pagi': 2, 'siang': 3, 'malam': 4}

In [35]:
df_patterns['labels'] = df_patterns['tag'].map(lambda x: label2id[x.strip()])
df_patterns.head(20)

,tag,patterns,labels
0,salam,hai,0
1,salam,halo,0
2,salam,hola,0
3,salam,helo,0
4,salam,hallo,0
5,salam,hello,0
6,salam,ola,0
7,tanya keberadaan,ada orang sana,1
8,tanya keberadaan,orang sana,1
9,tanya keberadaan,orang ga sana,1


# Training

## Split data into train and test

In [36]:
X = list(df_patterns['patterns'])
X[:5]

['hai', 'halo', 'hola', 'helo', 'hallo']

In [37]:
y = list(df_patterns['labels'])
y[:5]

[0, 0, 0, 0, 0]

In [38]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state = 123, test_size=0.2)

## BERT Model

### Tokenize

In [39]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification

model_name='cahya/bert-base-indonesian-522M'
tokenizer = BertTokenizer.from_pretrained(model_name)
tokenizer.save_pretrained("./model-fine-tuned-1/")

c:\laragon\bin\python\python-3.10\lib\site-packages\tqdm\auto.py:21: TqdmWarning:

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html



('./model-fine-tuned-1/tokenizer_config.json',
 './model-fine-tuned-1/special_tokens_map.json',
 './model-fine-tuned-1/vocab.txt',
 './model-fine-tuned-1/added_tokens.json')

In [40]:
model = BertForSequenceClassification.from_pretrained(
        model_name,
        num_labels=num_labels,
        id2label=id2label,
        label2id = label2id,
        hidden_dropout_prob=0.1,
        attention_probs_dropout_prob=0.25
    )
model.save_pretrained("./model-fine-tuned-1/")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cahya/bert-base-indonesian-522M and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [41]:
train_encoding = tokenizer(X_train, truncation=True, padding=True)
test_encoding = tokenizer(X_test, truncation=True, padding=True)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [42]:
full_data = tokenizer(X, truncation=True, padding=True)

### Build data loader

In [43]:
from torch.utils.data import Dataset
class DataLoader(Dataset):

    def __init__(self, encodings, labels):

        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):

        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):

        return len(self.labels)

In [44]:
train_dataloader = DataLoader(train_encoding, y_train)
test_dataloader = DataLoader(test_encoding, y_test)

In [45]:
fullDataLoader = DataLoader(full_data, y_test)

### Define Evaluation Metrics

In [46]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted', labels=np.unique(y_test))
    acc = accuracy_score(labels, preds)

    return {
        'Accuracy': acc,
        'F1': f1,
        'Precision': precision,
        'Recall': recall
    }

### Hyper Parameter

In [47]:
num_epochs = 40

### Define Optimizer

In [48]:
from transformers import AdamW, get_scheduler
optimizer = AdamW(model.parameters(), lr=5e-5)
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

c:\laragon\bin\python\python-3.10\lib\site-packages\transformers\optimization.py:411: FutureWarning:

This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning



### Training Arguments

In [50]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm_probability=0.15)

In [51]:
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(
    output_dir='output',
    overwrite_output_dir=True,
    do_train=True,
    do_eval=True,
    num_train_epochs=num_epochs,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=16,
    warmup_steps=100,
    weight_decay=0.01,
    logging_strategy='steps',
    logging_dir='multi-class-logs',
    logging_steps=50,
    evaluation_strategy="steps",
    eval_steps=100,
    save_strategy="steps",
    load_best_model_at_end=True
)

### Train the model

In [55]:
from transformers import EarlyStoppingCallback
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataloader,
    eval_dataset=test_dataloader,
    compute_metrics= compute_metrics,
    # callbacks = [EarlyStoppingCallback(early_stopping_patience=3)],
    optimizers=(optimizer, lr_scheduler)
)

In [56]:
trainer.train()

                                                   
  1%|          | 50/5800 [01:37<3:02:57,  1.91s/it]

{'loss': 6.3633, 'learning_rate': 4.9986460138648184e-05, 'epoch': 0.34}


                                                    
  2%|▏         | 100/5800 [03:23<3:19:57,  2.10s/it]

{'loss': 5.9698, 'learning_rate': 4.9972920277296366e-05, 'epoch': 0.69}










































































c:\laragon\bin\python\python-3.10\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

                                                    
                                     

  2%|▏         | 100/5800 [03:39<3:19:57,  2.10s/it]



...









































































c:\laragon\bin\python\python-3.10\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

                                                      
                                        

100%|██████████| 5800/5800 [18:18:31<00:00,  1.93s/it]



{'eval_loss': 0.17118366062641144, 'eval_Accuracy': 0.9610051993067591, 'eval_F1': 0.9590083804121934, 'eval_Precision': 0.9710427498555747, 'eval_Recall': 0.9610051993067591, 'eval_runtime': 16.0425, 'eval_samples_per_second': 71.934, 'eval_steps_per_second': 4.55, 'epoch': 40.0}


                                                      
100%|██████████| 5800/5800 [18:18:32<00:00, 11.36s/it]

{'train_runtime': 65912.763, 'train_samples_per_second': 2.801, 'train_steps_per_second': 0.088, 'train_loss': 0.46190192773424343, 'epoch': 40.0}


TrainOutput(global_step=5800, training_loss=0.46190192773424343, metrics={'train_runtime': 65912.763, 'train_samples_per_second': 2.801, 'train_steps_per_second': 0.088, 'train_loss': 0.46190192773424343, 'epoch': 40.0})

### Evaluate model

In [57]:
q=[trainer.evaluate(eval_dataset=df2) for df2 in [train_dataloader, test_dataloader]]

pd.DataFrame(q, index=["train","test"]).iloc[:,:5]

100%|██████████| 73/73 [00:16<00:00,  3.72it/s]c:\laragon\bin\python\python-3.10\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

100%|██████████| 73/73 [00:16<00:00,  4.42it/s]


,eval_loss,eval_Accuracy,eval_F1,eval_Precision,eval_Recall
train,0.022804,0.987652,0.988094,0.991118,0.989205
test,0.157999,0.962738,0.961991,0.974798,0.962738


In [58]:
def predict(text):

    inputs = tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors="pt").to("cuda")
    outputs = model(**inputs)

    probs = outputs[0].softmax(1)
    pred_label_idx = probs.argmax()
    pred_label = model.config.id2label[pred_label_idx.item()]

    return probs, pred_label_idx, pred_label

In [59]:
text = "selamat pagi"
predict(text)

(tensor([[8.0069e-06, 4.1251e-06, 9.9934e-01, 2.4759e-05, 2.6664e-05, 1.1428e-05,
          3.6877e-07, 4.3578e-07, 4.7828e-07, 4.9260e-06, 4.5316e-06, 1.8492e-06,
          7.0150e-07, 6.6077e-07, 1.0512e-06, 4.3449e-07, 1.8784e-06, 2.7948e-07,
          9.0446e-07, 1.2383e-06, 1.7999e-06, 1.0734e-06, 4.9323e-06, 5.9398e-07,
          3.7069e-06, 1.5958e-06, 6.8531e-07, 9.4416e-07, 3.2566e-07, 3.1045e-06,
          3.5348e-06, 3.9760e-07, 1.8224e-06, 1.3994e-06, 4.3560e-06, 1.3833e-06,
          1.2254e-06, 5.7374e-07, 1.2913e-06, 4.0154e-07, 1.1501e-07, 3.1097e-05,
          1.6986e-06, 9.7104e-07, 2.9298e-07, 3.0160e-06, 1.1728e-07, 1.6579e-07,
          7.8910e-08, 1.0445e-07, 1.0844e-06, 9.1306e-08, 2.9002e-07, 3.5791e-07,
          5.6079e-07, 4.2487e-07, 2.5410e-06, 9.5250e-07, 4.0907e-07, 4.4895e-07,
          3.0858e-07, 1.3944e-06, 4.2076e-07, 3.5256e-07, 3.3055e-06, 5.1872e-07,
          3.8324e-07, 3.2001e-07, 1.2075e-06, 4.4573e-08, 1.7416e-07, 4.7920e-07,
          1.8303

### Save model

In [60]:
model_path = "chatbot"
trainer.save_model(model_path)
tokenizer.save_pretrained(model_path)

('chatbot\\tokenizer_config.json',
 'chatbot\\special_tokens_map.json',
 'chatbot\\vocab.txt',
 'chatbot\\added_tokens.json')

In [2]:
from transformers import BertTokenizerFast, BertForSequenceClassification
from transformers import pipeline
model_path = "chatbot"

model = BertForSequenceClassification.from_pretrained(model_path)
tokenizer= BertTokenizerFast.from_pretrained(model_path)
chatbot= pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

In [3]:
chatbot("halo")

[{'label': 'salam', 'score': 0.9991292357444763}]

### Chat with Chatbot

In [63]:
def preprocessing_user_input(text):
    text = lower(text)
    text = non_ascii(text)
    text = remove_emoji(text)
    text = remove_tab(text)
    text = remove_tab2(text)
    text = remove_punctuation(text)
    text = remove_excessive_dot(text)
    text = remove_angka(text)
    text = remove_whitespace_LT(text)
    text = remove_whitespace_multiple(text)
    text = replace_slang_word(text)
    text = stopwords(text)
    text = stemming(text)
    return text

def chat(chatbot):

    print("Chatbot: Hi! Saya adalah asisten virtual Anda, Jangan ragu untuk bertanya, dan saya akan melakukan yang terbaik untuk memberi Anda jawaban dan bantuan..")
    print("Ketik 'quit' untuk keluar dari chat\n\n")

    text = input("User: ").strip()

    text = preprocessing_user_input(text)

    while(text != 'quit'):

        score = chatbot(text)[0]['score']

        # if score < 0.5:
        #   print(score)
        #   print("Chatbot: Maaf, saya tidak bisa menjawabnya\n\n")
        #   text = input("User: ").strip().lower()
        #   continue

        label = label2id[chatbot(text)[0]['label']]
        response = random.choice(intents['intents'][label]['responses'])

        print(text)
        print(score)
        print(f"Chatbot: {response}\n\n")

        text = input("User: ").strip().lower()
        text = preprocessing_user_input(text)


In [64]:
chat(chatbot)

Chatbot: Hi! Saya adalah asisten virtual Anda, Jangan ragu untuk bertanya, dan saya akan melakukan yang terbaik untuk memberi Anda jawaban dan bantuan..
Ketik 'quit' untuk keluar dari chat




In [1]:
import streamlit as st
import random
import time

st.title("Chatbot")

ImportError: cannot import name 'title' from partially initialized module 'streamlit' (most likely due to a circular import) (c:\Code\py\chatbot-mental-health\streamlit.py)